In [1]:
from tensorflow import keras

In [2]:
#Норм 0 фокус 1 засвет 2 движение 3
model=keras.Sequential([
    keras.layers.SimpleRNN(5,input_shape=[None,5],activation="tanh",return_sequences=True),
    keras.layers.SimpleRNN(100,input_shape=[None,5],activation="tanh",return_sequences=True),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(4,activation="softmax")
])
model.compile(loss="spare_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 5)           55        
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 100)         10600     
                                                                 
 dense (Dense)               (None, None, 50)          5050      
                                                                 
 dense_1 (Dense)             (None, None, 4)           204       
                                                                 
Total params: 15909 (62.14 KB)
Trainable params: 15909 (62.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
